<a href="https://colab.research.google.com/github/akaraci/COVID-19/blob/master/VGGCOV19_NET_and_VGG19_Binary_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autotime
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics
from google.colab import drive
drive.mount('/content/drive')


import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.callbacks import EarlyStopping
np.random.seed(1000)

from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import  ImageDataGenerator, img_to_array, load_img
from glob import glob

In [ ]:
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum.pickle", "rb"))
X = X/255.0 
y = to_categorical(y, num_classes = 3)
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    print(x_train.shape)
    print(x_test.shape)


In [ ]:
#VGGCOV19-NET Binary Class Code
%load_ext autotime
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum_two.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum_two.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 2)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer'ı iptal eder
vgg_layer_list=vgg.layers
model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)

for layer in model.layers:
    layer.trainable=False

numberOfClass=2
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(256))
model.add(Dense(512))
model.add(Dense(numberOfClass,activation='softmax'))

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
aucs = []
tprs = []
fprs = []
base_fpr = np.linspace(0, 1, 101)
history=[]
acc_val=[]
result=[]
plt.figure(figsize=(10, 10))
fold=1
opt=keras.optimizers.Adam(lr=0.001)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    
    model.compile(loss="categorical_crossentropy", optimizer=opt,  metrics=["accuracy"])
    history = model.fit(x_train, y_train, batch_size=32,epochs=100, verbose=0) #validation_split=0.2   validation_data=(x_test, y_test)
    
    np.save("drive/My Drive/Datacovid_kemal/modelsmodifytwo/my_historyw"+str(fold)+".npy",history.history)
    
    result.append(model.predict(x_test))
    prediction = model.predict(x_test)
    Y_pred_classes = np.argmax(prediction,axis = 1) 
    y_test_classes = np.argmax(y_test,axis = 1)
    print(metrics.confusion_matrix(y_test_classes, Y_pred_classes))
    print(metrics.classification_report(y_test_classes, Y_pred_classes, digits=2))
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1])
    print(conf_mat)
    total1=sum(sum(conf_mat))
    accuracy1=(conf_mat[0,0]+conf_mat[1,1])/total1
    acc_val.append(accuracy1)
    print ('Accuracy : ', accuracy1)
    probas_ = model.predict_proba(x_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test_classes, probas_[:, 1],drop_intermediate=False,pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    print("Auc-"+str(fold)+"=",roc_auc)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC Fold- %d (AUC = %0.2f)' % (fold, roc_auc))
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)
    fold=fold+1
mean_auc=np.mean(aucs)
print("Mean AUC:",mean_auc)
mean_acc=np.mean(acc_val)
print("Mean ACC:",mean_acc)
tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
mean_auc1 = metrics.auc(base_fpr, mean_tprs)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std
std_auc = np.std(aucs)
plt.plot(base_fpr, mean_tprs, 
          label='Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
          lw=2, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)    
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Cross-Validation ROC of Model-4 ')
plt.legend(loc="lower right", prop={'size': 15})
plt.show()


In [ ]:
#VGG19-NET Binary Class Code
%load_ext autotime
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum_two.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum_two.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 2)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer canceled

model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)


for layer in model.layers:
    layer.trainable=False

numberOfClass=2
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Dense(numberOfClass,activation='softmax'))

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
aucs = []
tprs = []
fprs = []
base_fpr = np.linspace(0, 1, 101)
history=[]
acc_val=[]
result=[]
plt.figure(figsize=(10, 10))
fold=1
opt=keras.optimizers.Adam(lr=0.001)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    
    model.compile(loss="categorical_crossentropy", optimizer=opt,  metrics=["accuracy"])
    history = model.fit(x_train, y_train, batch_size=32,epochs=100, verbose=0) 
    
    np.save("drive/My Drive/Datacovid_kemal/modelsmodifytwo/my_historyw"+str(fold)+".npy",history.history)
    
    result.append(model.predict(x_test))
    prediction = model.predict(x_test)
    Y_pred_classes = np.argmax(prediction,axis = 1) 
    y_test_classes = np.argmax(y_test,axis = 1)
    print(metrics.confusion_matrix(y_test_classes, Y_pred_classes))
    print(metrics.classification_report(y_test_classes, Y_pred_classes, digits=2))
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1])
    print(conf_mat)
    total1=sum(sum(conf_mat))
    accuracy1=(conf_mat[0,0]+conf_mat[1,1])/total1
    acc_val.append(accuracy1)
    print ('Accuracy : ', accuracy1)
    probas_ = model.predict_proba(x_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test_classes, probas_[:, 1],drop_intermediate=False,pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    print("Auc-"+str(fold)+"=",roc_auc)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC Fold- %d (AUC = %0.2f)' % (fold, roc_auc))
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)
    fold=fold+1
mean_auc=np.mean(aucs)
print("Mean AUC:",mean_auc)
mean_acc=np.mean(acc_val)
print("Mean ACC:",mean_acc)
tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
mean_auc1 = metrics.auc(base_fpr, mean_tprs)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std
std_auc = np.std(aucs)
plt.plot(base_fpr, mean_tprs, 
          label='Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
          lw=2, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)    
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Cross-Validation ROC of Model-4 ')
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
pip install scikit-plot

In [ ]:
!pip install ipython-autotime

In [ ]:
#VGGCOV19-NET Summary

X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum_two.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum_two.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 2)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer'ı iptal eder
vgg_layer_list=vgg.layers
model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)
#print(model.summary())

for layer in model.layers:
    layer.trainable=False

numberOfClass=2
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(256))
model.add(Dense(512))
model.add(Dense(numberOfClass,activation='softmax'))
print(model.summary())

In [ ]:
#VGG19 Summary
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum_two.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum_two.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 2)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer'ı iptal eder

model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)
#print(model.summary())

for layer in model.layers:
    layer.trainable=False

numberOfClass=2
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Dense(numberOfClass,activation='softmax'))
print(model.summary())
